Welcome to the SQL mini project. You will carry out this project partly in
the PHPMyAdmin interface, and partly in Jupyter via a Python connection.

This is Tier 2 of the case study, which means that there'll be less guidance for you about how to setup
your local SQLite connection in PART 2 of the case study. This will make the case study more challenging for you: 
you might need to do some digging, and revise the Working with Relational Databases in Python chapter in the previous resource.

Otherwise, the questions in the case study are exactly the same as with Tier 1. 

PART 1: PHPMyAdmin
You will complete questions 1-9 below in the PHPMyAdmin interface. 
Log in by pasting the following URL into your browser, and
using the following Username and Password:

URL: https://sql.springboard.com/  
Username: student  
Password: learn_sql@springboard

The data you need is in the "country_club" database. This database
contains 3 tables:  
    i) the "Bookings" table,  
    ii) the "Facilities" table, and  
    iii) the "Members" table.

In this case study, you'll be asked a series of questions. You can
solve them using the platform, but for the final deliverable,
paste the code for each solution into this script, and upload it
to your GitHub.

Before starting with the questions, feel free to take your time,
exploring the data, and getting acquainted with the 3 tables. 



# **PART I**
##                                  **QUESTIONS** 

**Q1:** Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do. 

In [ ]:
SELECT name 
    FROM Facilities 
    WHERE membercost > 0;

**Q2:** How many facilities do not charge a fee to members?

In [ ]:
SELECT COUNT(name) 
    FROM Facilities 
    WHERE membercost = 0;

**Q3:** Write an SQL query to show a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question. 

In [ ]:
SELECT facid, name, membercost, monthlymaintenance 
       FROM Facilities 
       WHERE membercost < (monthlymaintenance * 0.2); 

**Q4:** Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator.

In [ ]:
SELECT * 
    FROM Facilities 
    WHERE facid IN (1,5);

**Q5:** Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question. 

In [ ]:
SELECT name, monthlymaintenance, 
    CASE 
        WHEN monthlymaintenance > 100 THEN 'expensive' 
        ELSE 'cheap' 
        END AS COE 
        FROM Facilities; 

**Q6:** You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution. 

In [ ]:
SELECT firstname, surname 
    FROM Members 
    ORDER BY joindate DESC;

**Q7:** Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name. 

In [ ]:
SELECT DISTINCT concat(firstname,' ', surname) AS Names, F.name AS Facility 
    FROM Members AS M 
    INNER JOIN Bookings AS B 
    ON M.memid = B.memid 
    INNER JOIN Facilities AS F 
    ON B.facid = F.facid 
    WHERE B.facid IN (0, 1);

**Q8:** Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries.

In [ ]:
SELECT F.name AS Facility,
    CASE 
        WHEN B.memid = 0 THEN 'Guest' 
        ELSE concat(firstname,' ',surname) 
        END AS Names,
    CASE 
	    WHEN B.memid = 0 THEN F.guestcost
        ELSE F.membercost
        END AS Cost 
    FROM Members AS M 
    INNER JOIN Bookings AS B 
    ON M.memid = B.memid 
    INNER JOIN Facilities AS F 
    ON B.facid = F.facid 
    WHERE starttime LIKE '2012-09-14%' 
    AND ( 
        (B.memid != 0 AND membercost > 30) OR (B.memid = 0 AND guestcost > 30) 
        )
    ORDER BY Cost DESC;

**Q9:** This time, produce the same result as in Q8, but using a subquery.

In [ ]:
SELECT F.name AS Facility, concat(firstname,' ',surname),
	CASE 
	    WHEN B.memid = 0 AND F.guestcost > 30 THEN F.guestcost
        WHEN B.memid != 0 AND F.membercost > 30 THEN F.membercost
        END AS Cost 
    FROM Members AS M 
    INNER JOIN Bookings AS B 
    ON M.memid = B.memid 
    INNER JOIN Facilities AS F 
    ON B.facid = F.facid 
    WHERE starttime IN (
    	SELECT starttime
        FROM Bookings
        WHERE starttime LIKE '2012-09-14%') 
    AND ( 
        (B.memid != 0 AND membercost > 30) OR (B.memid = 0 AND guestcost > 30) 
        )
    ORDER BY Cost DESC;

# **PART 2: SQLite**

Export the country club data from PHPMyAdmin, and connect to a local SQLite instance from Jupyter notebook for the following questions.  

In [1]:
from sqlalchemy import create_engine
import pandas as pd
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')

## **QUESTIONS**
**Q10:** Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!

In [6]:
pd.read_sql_query('''SELECT F.name, F.guestcost * (SELECT COUNT(bookid) 
                                                   FROM Bookings AS B 
                                                   WHERE memid = 0 AND B.facid = F.facid) +
                                    membercost * (SELECT COUNT(bookid) 
                                                  FROM Bookings AS B 
                                                  WHERE memid != 0 AND B.facid = F.facid) AS Revenue
                     FROM Facilities AS F 
                     INNER JOIN Bookings AS B
                     GROUP BY name
                     Having Revenue < 1000
                                ''', engine)

,name,Revenue
0,Badminton Court,604.5
1,Pool Table,265.0
2,Snooker Table,115.0
3,Table Tennis,90.0


**Q11:** Produce a report of members and who recommended them in alphabetic surname, firstname order

In [29]:
pd.read_sql_query('''SELECT firstname || ' ' || surname AS Member, (SELECT firstname || ' ' || surname
                                                                    FROM Members
                                                                    WHERE)  
                     FROM Members
                     ORDER BY surname, firstname
                     LIMIT 5;
                    ''', engine)

,Member,(SELECT firstname || ' ' || surname\n FROM Members\n WHERE recommendedby = memid)
0,Florence Bader,None
1,Anne Baker,None
2,Timothy Baker,None
3,Tim Boothe,None
4,Gerald Butters,None


**Q12:** Find the facilities with their usage by member, but not guests

In [33]:
pd.read_sql_query('''SELECT F.name Facility, COUNT(bookid) USAGE
                     FROM Bookings AS B
                     INNER JOIN Members AS M
                     USING (memid)
                     INNER JOIN Facilities AS F
                     USING (facid)
                     WHERE memid != 0
                     GROUP BY F.name; ''', engine)

,Facility,USAGE
0,Badminton Court,344
1,Massage Room 1,421
2,Massage Room 2,27
3,Pool Table,783
4,Snooker Table,421
5,Squash Court,195
6,Table Tennis,385
7,Tennis Court 1,308
8,Tennis Court 2,276


**Q13:** Find the facilities usage by month, but not guests

In [35]:
pd.read_sql_query('''SELECT F.name Facility, COUNT(bookid) USAGE
                     FROM Bookings AS B
                     INNER JOIN Members AS M
                     USING (memid)
                     INNER JOIN Facilities AS F
                     USING (facid)
                     WHERE memid != 0
                     GROUP BY F.name, STRFTIME("%m-%Y", starttime); ''', engine)

,Facility,USAGE
0,Badminton Court,51
1,Badminton Court,132
2,Badminton Court,161
3,Massage Room 1,77
4,Massage Room 1,153
5,Massage Room 1,191
6,Massage Room 2,4
7,Massage Room 2,9
8,Massage Room 2,14
9,Pool Table,103
